## Projeto 2: Engenharia de Conhecimento 20/21

## O Caça-Fantasma 

##### Lançamento: 4 de Abril de 2021 ao fim do dia
### ***Entrega***: 18 de Abril de 2021 às 23h59

<img src="files/imagens/cacaFantasmas.jpg" alt="Drawing" style="width: 200px;"/> 

## Introdução

No jogo Caça Fantasma temos uma grelha discreta formada por $N$x$M$ células, numa das quais o fantasma está escondido, mas não sabemos qual - pode ser qualquer uma delas com igual probabilidade, ele não tem quaisquer preferências. Temos a possibilidade de fazer uso de sensores que sofrem da doença da incerteza e cujas cores nos ajudam a dar uma indicação sobre a posição mais provável do fantasma. Para isso, fazemos uso de um modelo de distribuição dos sensores dada a distância ao fantasma, o qual usamos para fazer observações. Para escolher a casa mais provável para o fantasma, faremos uso de um modelo do jogo implementado numa Rede de Bayes e, recorrendo aos mecanismos de inferência nas Redes de Bayes, poderemos obter a probabilidade de ele se esconder em cada uma das células, dadas as cores observadas qu7andoi apontamos os sensores a algumas células.  Vamos assumir que no máximo poderemos fazer $C$ observações ao acaso, i.e. podemos ler os valores dos sensores em $C$ casas diferentes e depois teremos de tentar acertar na posição do fantasma, escolhendo aquela com maior probabilidade, e desempatando de forma aleatória.

<img src="files/imagens/cacaFantasmas.PNG" alt="Drawing" style="width: 200px;"/> 
<p style="text-align: center;">Numa grelha $4$x$6$ acertámos no fantasma após observar 4 casas.</p>


Notem que não há qualquer uso de inteligência na escolha das $C$ diferentes casas para observar com os respectivos sensores. Elas serão escolhidas completamente ao acaso. No fundo queremos responder à pergunta seguinte: Se tivermos uma grelha de $N$x$M$ após $C$ observações aleatórias, qual a taxa média de sucesso e de que modo cresce com o aumento do númewro de observações?

### Representação do espaço
Num espaço rectangular, formado por $N$ linhas e $M$ colunas, considerem que a casa de topo esquerdo é a casa $(1,1)$ e que a casa de fundo à direita é a casa $(N,M)$. Na imagem em cima, o fantasma estará na casa $(4,6)$ e as casas observadas são $(3,3)$-sensor a verde, $(3,4)$-sensor com a cor amarela, $(4,5)$=sensor indica laranja e $(4,6)$-sensor indica encarnado. 

### Modelo dos sensores
Como foi referido atrás, cada sensor pode ter diferentes cores: `encarnado`, `laranja`, `amarelo` e `verde`, mas de acordo com uma distribuição de probabilidades que depende da distância à posição do fantasma. Eis um exemplo de um modelo dos sensores: 

| Dist | enc. | lar. | amar. | ver. |
|------|-----------|---------|---------|-------|
|  0   |   0.80    |   0.15  |   0.04  |  0.01 |
|  1   |   0.10    |   0.60  |   0.25  |  0.05 |
|  2   |   0.10    |   0.60  |   0.25  |  0.05 |
|  3   |   0.05    |   0.15  |   0.50  |  0.30 |
|  4   |   0.05    |   0.15  |   0.50  |  0.30 |
|  5   |   0.01    |   0.05  |   0.30  |  0.60 |
| $\ge$ 6  |   0.01   |   0.05   |   0.30   |   0.60   |

Usaremos `modeloSensor` que é uma variável global Python, que é um dicionário, com o modelo dos sensores que corresponde à tabela em cima, em que as chaves são inteiros (distâncias ao fantasma) e os valores são distribuições de probabiidade.

```python
"""
modeloSensor[t] é uma distribuição de probabilidades dada a distância t (ao fantasma)
modeloSensor[1]['a'], por exemplo, é a probabilidade que o sensor indique amarelo
dada a distância de 1 ao fantasma.
"""

modeloSensor = {
  0: {'e': .8, 'l': .15, 'a':0.04, 'v':0.01},
  1: {'e': .1, 'l': .60, 'a':0.25, 'v':0.05},
  2: {'e': .1, 'l': .60, 'a':0.25, 'v':0.05},
  3: {'e': .05, 'l': .15, 'a':0.5, 'v':0.3},
  4: {'e': .05, 'l': .15, 'a':0.5, 'v':0.3},
  5: {'e': .01, 'l': .05, 'a':0.3, 'v':0.6},
  6: {'e': .01, 'l': .05, 'a':0.3, 'v':0.6}}
```


### Modelização através de uma Rede de Bayes

Eis a estrutura da rede de Bayes correspondente ao jogo do Caça Fantasma para uma grelha $4$x$4$ que é formada por 17 nós, com as variáveis:

$$\{F, S1\_1, S1\_2, S1\_3, S1\_4, S2\_1,S2\_2,S2\_3, S2\_4,S3\_1, S3\_2, S3\_3, S3\_4,S4\_1, S4\_2, S4\_3, S4\_4\}$$.

<img src="files/imagens/redeBayesCacaFantasma4x4.PNG
" alt="Drawing" style="width: 250px;"/> 
<p style="text-align: center;">A Rede de Bayes para o Caça Fantasma numa grelha $4$x$4$.</p>


Notem que a variável $F$ para uma dimensão $4$x$4$ terá que ter um domínio de 16 células e que cada variável do tipo sensor terá que ter um domínio de 4 valores, representando as 4 cores, independentemente das dimensões do espaço. No caso de termos um espaçço $N$x$M$ a rede Bayes terá uma variável para o fantasma e $N$x$M$ variáveis para os sensores.

Será inevitável usar uma Rede de Bayes não booleana que se adapte ao problema do Caça Fantasmas para grelhas com diferentes dimensões.


### Classes a redefinir

#### NoBayes e RedeBayes 
Na aula prática 3 apresentámos as classes `BayesNet` e `BayesNode` (incluídas no ficheiro probabilityPlus.py) que estão limitadas a implementar Redes de Bayes binárias. Para modelizarem o Caça Fantasmas para lá de um espaço com duas casas, precisarão de adaptar essas classes criando duas novas classes: `RedeBayes`e `NoBayes` de modo a terem Redes de Bayes com nós com variáveis com quaisquer domínios.

Essas classes terão que manter a possibilidade das várias formas de introduzir os dados, como por exemplo podermos introduzir apenas as probabilidades condicionais para o valor True de variáveis booleanas. 


**Atenção**: Notem que as `cpts` das Redes de Bayes na classe `BayesNode` são representadas por um dicionário e que o formato das chaves correspondem a tuplos com os valores das variáveis dos nós pais. Assim, existirá necessariamente um conflito se representarem os valores da variável $F$ como tuplos com coordenadas $(L,C)$. Por exemplo, o tuplo $(1,1)$ na chave da cpt de qualquer das variáveis dos sensores $SL\_C$, por exemplo, será interpretado como uma entrada de uma tabela com duas variáveis pais, ambas com o valor $1$.
Assim, nesse caso, o domínio de $F$ não deve ser formado por duplos mas strings com duplos:

$Dom(F)=\{\ '(1,1)\ ',\ '(1,2)\ ',\ '(1,3)\ ',\ '(1,4)\ ',\ '(2,1)\ ',\ '(2,2)\ ',\ '(2,3)\ ',\ '(2,4)\ ',\ '(3,1)\ ',(3,2)\ ',(3,3)\ ',\ '(3,4)\ ',\ '(4,1)\ ',\ '(4,2)\ ',\ '(4,3)\ ',\ '(4,4)\ '\}$

As variáveis dos sensores $SL\_C$ para qualquer linha $L$ e coluna $C$ terão como domínios:


$Dom(SL\_C)=\{\ 'e\ ',\ 'l\ ',\ 'a\ ',\ 'v\ '\}$

Sabemos que a probabilidade apriori da posição do fantasma $P(F)$ segue uma distribuição uniforme e que as tabelas de probabilidades condicionais de cada uma das variáveis correspondentes aos sensores segue o modelo representado na variável global `modeloSensor` e que depende da distância de Manhatan entre o valor da posição do fantasma e a célula respectiva da variável.

Notem que os valores inseridos, para as `cpt`s das variaveis não booleanas, não têm de estar normalizados, tendo que ficar normalizados após a construção das instâncias das classe `RedeBayes` e `NoBayes` (ver exemplos de teste a seguir).

#### Exemplos de Redes a serem usadas em teste:

```python
T, F = True, False

burglary = RedeBayes([
    ('Burglary', '', 0.001),
    ('Earthquake', '', {"strong":1,"light":10,"none":89 }),
    ('Alarm', 'Burglary Earthquake',
     {(T, T): 0.95, (T, F): 0.94, (F, T): 0.29, (F, F): 0.001}),
    ('JohnCalls', 'Alarm', {T: 0.90, F: 0.05}),
    ('MaryCalls', 'Alarm', {T: 0.70, F: 0.01})
])

flu_net = RedeBayes([
            ('Vaccinated', '', 0.001),
            ('Flu', 'Vaccinated', {T: 0.002, F: 0.02}),
            ('Fever', 'Flu', {T:{'no':25, 'mild':25, 'high':50},F:{'no':97, 'mild':2,  'high':1}}),
            ('Headache', 'Flu', {T: 0.5,   F: 0.03})])

on_the_road = RedeBayes([
            ('P', '', 0.60),
            ('R', '', 0.20),
            ('V', 'P', {T:{'v1':0.2, 'v2':0.5, 'v3':0.3},F:{'v1':0.6, 'v2':0.3,'v3':0.1}}),
            ('M', 'V R', {('v1',T):0,('v2',T):0.6,('v3',T):0.9,('v1',F):0,('v2',F):0,('v3',F):0})])
```

### Funções a desenvolver para o projecto

#### cassa_fantasma
A função `cassa_fantasma` que recebe como input, por esta ordem, o número de $N$ linhas, o número de $M$ colunas e o modelo dos sensores e devolve uma Rede de Bayes com uma variável $F$ que tem como domínio as casas da grelha e com $N$x$M$ variáveis $SN\_M$ que correspondem aos sensores de cada uma das casas e que são todos equivalentes. Quaisquer destas variáveis pertencem a nós que possuem o nó com a variável $F$ como pai (ver figura em cima de uma Rede de Bayes para um espaço $4$x$4$). Assim, para um espaço $N$x$M$ teremos uma rede com $NxM+1$ variáveis.
Ordenem as variáveis: primeiro $F$ e depois as respeitantes aos sensores, ordenando pelas linhas e depois pelas colunas. Por exemplo, se tivessemos um espaço $4$x$6$ teríamos a sequência de $S1\_1$ até $S1\_6$, depois de $S2\_1$ até $S2\_6$, até finalmente $S4\_6$.

#### cassando_o_fantasma
Queremos que desenvolvam a função booleana `cassando_o_fantasma` que recebe a rede de bayes, a posição real do fantasma, e um conjunto de evidência correspondente aos valores lidos de alguns dos sensores, por esta ordem, e que devolve a indicação se encontrou ou não o fantasma. O caçador de fantasma escolhe sempre a casa mais provável para o fantasma dada a evidência e pode desempatar de forma aleatória, que é depois comparada com a casa onde ele está realmente, input do problema.
Notem que para calcularem as probabilidades de o fantasma estar em cada uma das células, podem usar o método `enumeration-ask` ou o método alternativo `elimination_ask`, que implementam a inferência nas Redes de Bayes. Notem que o método `elimination_ask` não foi dado nesta disciplina mas se verificarem que é mais rápido pode ser utilizado como uma caixa negra. De um modo semelhante à avaliação 2, podem em alternativa usar a Regra de Bayes para simplificar ainda mais o cálculo das probabilidades condicionais, e calcularem depois a casa mais provável para o fantasma dada a evidência.


#### tiro_no_fantasma
Façam uma versão 'pretty' do `cassando_fantasma`, chamada de `tiro_no_fantasma` que ilustre o espaço do caça fantasmas e as cores conhecidas dos sensores que fazem parte da evidência, bem como a célula prevista para o fantasma e a indicação de sucesso ou de falhanço.
São livres de fazerem o `pretty display` do modo que quiserem, mas damos uma sugestão de baixa pixelagem

##### Exemplo

Imaginemos que temos um espaço $2$x$2$ e que conhecemos os valores dos sensores da casa $(1,2)$, laranja e da casa $(2,2)$, amarelo.

Suponhamos que a casa mais provável para o fantasma é a $(1,1)$ e que o fantasma está mesmo em $(1,1)$

```python
Evidência:
    . l
    . a

Tiro no fantasma!
    $ l
    . a
    ```

Para a mesma evidência, se a casa mais provável para o fantasma é a $(1,1)$ mas o fantasma está em $(1,2)$

```python
Evidência:
    . l
    . a

Água:
    ~ $
    . a
```


Podem usar cores mesmo para as casas ou outras formas, é uma função livre que deve ser "pretty".
Sabemos que podemos ter poder escolher as cores das letras em Markdown. Também podem imprimir as probabilidades arredondadas do fantasma estar em cada uma das posições. 

#### observa_casa
Queremos uma função para observar o valor do sensor de uma determinada casa quando o fantasma está numa determinada célula, de acordo com acordo com a distância entre as duas células e o respectivo modelo dos sensores. Esta função recebe como input, por esta ordem, a casa, a posição do fantasma e o modelo dos sensores, e devolve uma cor.

#### taxa_sucessos
Finalmente pedimos a função que, para um número N de observações aleatórias, uma instância de uma RedeBayes, um modelo de sensores e um número $R$ significativo de repetições, devolve a média da taxa de sucesso da caça ao fantasma.

#### Exemplos do cálculo das taxas médias de sucesso
Devem usar a função `taxa_sucessos` para diferentes dimensões e dentro de cada dimensão para a diversidade do número de observações, mostrando uma tabela com os resultados. Devem repetir as caçadas aleatórias ao fantasma um número significativo de vezes para cada situação, devendo indicar qual o parâmetro $R$ que utilizaram na função `taxa_sucessos` (por exemplo 1000).

## Entrega

### Código a entregar
Têm de ter o código num único ficheiro, chamado ***EC2021-cassaFantasma-XX.py*** ( XX corresponde ao número do grupo).
Notem que antes de entregarem veriquem que o ficheiro corre numa pasta com os ficheiros Python que nós fornecemos, mas que não devem ser entregues.

### Código a não ser alterado
**Não alterem** nem o ***utils.py*** nem o ***probabilityPlus.py*** e, mais uma vez, **não os devem submeter!** Iremos correr os mesmos 2 ficheiros para todos.

 ### Relatório
O  relatório é **obrigatório** e também é obrigatório que seja feito no formato Jupyter Notebook. 

**IMPORTANTE**. Qualquer trabalho que não tenha relatório (só o código) ou que não cumpra esse formato não é avaliado e tem 0 de nota. Não consideramos de modo algum que os **comentários**, que podem ser importantes para perceber o código, sejam considerados para o relatório!

Nós fornecemos um ficheiro esqueleto, ***ec2021_CassaFantasma_XX.ipynb***, (substituam XX pelo número do grupo). Não se esqueçam de preencher os nomes e números dos elementos do vosso grupo.

### Prazo
Submetam um único ficheiro zip chamado de ***ec2021_CassaFantasma_XX.zip*** com pelo menos os dois ficheiros ***ec2021_CassaFantasma_XX.py*** e ***ec2021_CassaFantasma_XX.ipynb*** até ao dia **18 de bril** às 23:59. Na pasta a zipar podem adicionar as imagens que quiserem, numa pasta chamada de ***imagens***.